In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
   from google.colab import drive

   drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
dftrain=pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')

In [16]:
dftrain.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [15]:
dftrain['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [14]:
mapping = {
    "negative": 0,
    "positive": 1
}
dftrain['sentiment'] = dftrain['sentiment'].map(mapping)

In [17]:
port_stem=PorterStemmer()

In [20]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)

  return stemmed_content

In [22]:
dftrain["stemmed"]=dftrain['review'].apply(stemming)

In [27]:
dftrain.to_csv('/content/drive/My Drive/IMDB Stemmed.csv')

In [23]:
X=dftrain['stemmed']
Y=dftrain['sentiment']

In [25]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

In [28]:
#vectorizer=TfidfVectorizer()
#Xtrain_feature=vectorizer.fit_transform(Xtrain)
#Xtest_feature=vectorizer.transform(Xtest)

In [33]:
#model training Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
clf=Pipeline([('vec',TfidfVectorizer()),('rfc',RandomForestClassifier(n_jobs=-1))])
clf.fit(Xtrain,Ytrain)

Pipeline(steps=[('vec', TfidfVectorizer()),
                ('rfc', RandomForestClassifier(n_jobs=-1))])

In [30]:
#evalution
from sklearn.metrics import classification_report
Ypred=clf.predict(Xtest)
print(classification_report(Ytest,Ypred))

              precision    recall  f1-score   support

           0       0.85      0.86      0.86      5000
           1       0.86      0.85      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [31]:
import pickle
pickle.dump(clf,open('/content/drive/My Drive/model.pkl','wb'))

In [34]:
clf.predict(['the moovie was full of suspense'])

array([1])